## Subgraphs

https://thegraph.com/hosted-service/subgraph/traderjoe-xyz/exchange

### TraderJoe

13/09/22

In [2]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [15]:
endpoint = 'https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange'

In [22]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
response = requests.post(endpoint, json={'query':query})
if response.ok:
    data = response.json()
else:
    print('Error')
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['_meta']['block']['timestamp'], base = 0)))
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-13 14:20:03


True

6-Month Average Daily Volume

In [25]:
query = """{
  dayDatas(first:183, orderBy: date, orderDirection:desc){
  id
  volumeUSD
    volumeAVAX
    txCount
    date
    liquidityUSD
    liquidityAVAX
  }
}"""

In [26]:
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()
else:
    print('Retry!')

In [30]:
df = pd.json_normalize(data['data']['dayDatas'])

In [32]:
df['volumeUSD'].astype(float).mean()

66445023.02619464

6-Month Average TVL

In [35]:
df['liquidityUSD'].astype(float).mean()

416099949.584649

Largest Pool Concentration

In [52]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
query{
pairs(first: 1000, where:{timestamp_gt: """+f'"{ts}"'+"""}, 
  orderDirection: asc, orderBy: timestamp){
    id
    name
    token0{
      symbol
      id
    }
    token1{
      symbol
      id
    }
    reserve0
    reserve1
    reserveUSD
    token0Price
    token1Price
    timestamp
    block
}
  }
"""
    request = requests.post(endpoint, json={'query':query})
    if request.ok:
        data = request.json()
        pool = pd.json_normalize(data['data']['pairs'])
        df = df.append(pool)
        df.drop_duplicates(inplace=True)
        if len(df) == sl:
            break
        ts = df['timestamp'].astype(int).max()

/tmp/ipykernel_118256/619727309.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_118256/619727309.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_118256/619727309.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_118256/619727309.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_118256/619727309.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_118256/619

In [59]:
df.reset_index(drop = True, inplace=True)

In [65]:
df['reserveUSD'].astype(float).nlargest(10)

9901     1.048430e+12
14911    3.005664e+09
16781    1.849206e+09
9720     2.682962e+08
14119    5.164998e+07
9725     3.582391e+07
1514     2.953435e+07
5337     2.565707e+07
4873     1.649611e+07
48       1.498821e+07
Name: reserveUSD, dtype: float64

In [68]:
df.drop(9901, axis = 0, inplace=True)

In [69]:
df

,id,name,reserve0,reserve1,reserveUSD,token0Price,token1Price,timestamp,block,token0.symbol,token0.id,token1.symbol,token1.id
0,0xe4b9865c0866346ba3613ec122040a365637fb46,WAVAX-USDT,66.655925611435778905,1353.151812,2667.979048581240155534706154927882,0.04925975416824537268180519570556508,20.30054791959632501661845815339531,1624941631,2504194,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,USDT,0xde3a24028580884448a5397872046a019649b084
1,0x43b9c8dec26c2d21146466caacabf94fdbeac473,WAVAX-DAI,2.843082470273760778,54.220307747745336624,108.5583935837497499030108330140589,0.05243574941516236122432337162354385,19.07095848068186927870679512710967,1624941782,2504226,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,DAI,0xba7deebbfc5fa1100fb055a87773e1e99cd3507a
2,0x454e67025631c065d3cfad6d71e6892f74487a15,JOE-WAVAX,17286792.793213274062898592,190579.524447463804592464,7607579.08983136112393525551898475,90.70645360949384693449353144990866,0.01102457388870216377837362093977407,1625028033,2516404,JOE,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
3,0xc992ab46428a5a2edeb8f44d946ce5642f97ef71,WAVAX-ETH,0.51308262619789782,0.005846738668896178,19.67197157622815917496675238570888,87.75535478050776489117986216710998,0.01139531601805021907293501104623983,1625029067,2516545,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,ETH,0xf20d962a6c8f70c731bd838a3a388d7d48fa6e15
4,0xbeb0b5fbd99b8e7498a4063cb419646922f6eef8,WBTC-WAVAX,0.00003315,0.047651557468385073,1.603806833616522987499175463578525,0.0006956750578822888668574859723863619,1437.452713978433574660633484162896,1625029156,2516553,WBTC,0x408d4cd0adb7cebd1f1a1c33a0ba2098e1295bab,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16977,0x2a8e370d15350eed9e930084f2db5b52f695fac9,MF-WAVAX,222964163080044.628366002372248143,90.089715690042090994,3843.56028018297751225882701475625,2474912495530.159407466129692274804,0.000000000000404054689531876400831646135288559,1662818624,19734132,MF,0x9a73840d55e265efcae53c2406dc89382378690f,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
16978,0xb887ce2409203c680b10aeda8986295f7204f8a6,WAVAX-AVL,0.013318812497170612,910895.410802632,0.5486529428923726249025847631461319,0.00000001462167043462738649184119999603524,68391638.59361624630852898363952464,1662858030,19746440,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,AVL,0xf49d79dd8367de01d506732eab569932ce0b3d77
16979,0x7aac10f6eaec2e997316f732ce30968323a6cb83,BLZ-USDC,277153.967832836,0.106247,0.2124966062899240210489721410455237,2608581.586612666710589475467542613,0.0000003833500953667830414821732602663874,1662858671,19746679,BLZ,0x66e19a8898d66858329027013bda0497a1dde38f,USDC,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
16980,0x996e16baaf3d39d4f5b4c7d5e96dcb6d865b1452,ACS-WAVAX,16053.873604120451098996,20.707328260426533629,825.8584480117593614981331062115924,775.2749848854605565720569863351826,0.001289864911799960125238402007441522,1662896239,19761258,ACS,0x18fc6360e83fe91404d47ea4400a221dfbbacf06,WAVAX,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7


In [70]:
df['reserveUSD'].astype(float).max()/df['reserveUSD'].astype(float).sum()

0.5518547613182163

In [71]:
df['reserveUSD'].astype(float).nlargest(10).sum()/df['reserveUSD'].astype(float).sum()

0.9746361473363759

Liquidity Pools with > $1mn TVL

In [72]:
len(df.loc[df['reserveUSD'].astype(float) > 1000000])

37

Monthly Active Users

Price impact

In [88]:
cs = df.loc[(df.index.isin(df['reserveUSD'].astype(float).nlargest(10).index)) & (df.index != 4873)].copy()

In [89]:
cs['reserveUSD'] = cs['reserveUSD'].astype(float)

In [93]:
1000/(cs['reserveUSD']/2).mean()

3.4014653975119434e-06

In [94]:
10000/(cs['reserveUSD']/2).mean()

3.4014653975119436e-05

In [95]:
100000/(cs['reserveUSD']/2).mean()

0.00034014653975119436